<a href="https://colab.research.google.com/github/flying-bear/AutoT/blob/master/3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание 3. Разрешение семантической многозначности.

При выполнении домашнего задания можно пользоваться тетрадками с семинаров.

### Описание задания:

Выберите одну из предложенных лексем: _свет_, _острый_, _лук_, _замок_.

1. Выберите 2-3 значения лексемы по словарю (рекомендую использовать [МАС](http://feb-web.ru/feb/mas/mas-abc/), обязательно напишите, каким словарем пользовались).
Для каждого значения придумайте несколько "диагностических" слов (которые сигнализируют о реализации одного из значений в контексте): 
например, _ключ_ — _вода_ / _дверь_. Подберите по НКРЯ контексты для выбранной лексемы, содержащие диагностические слова.
Таким образом, получается размеченный тестсет.

2. Для каждого контекста определите значение автоматически — с помощью AdaGram / ELMo на выбор.
В случае с AdaGram вы сразу получаете какой-то номер значения, с ELMo — нужно кластеризовать.

3. Возьмите словарные толкования для каждого значения и припишите ему значение автоматически
тем же способом, что и в п.2.

4. Придумайте и реализуйте способ соотнести толкования с значениями с помощью тезауруса (Wiki_ru_wordnet / РуТез). 

5. Оцените accuracy — в какой доле контекстов автоматически определенно значение для контекста совпадает 
с автоматически определенным значением толкования.

### Критерии оценки:

по 2 балла на каждый пункт

### Формат сдачи задания:

Jupyter-notebook на гитхабе.

### Дедлайн: 

9 декабря 2019 10:00мск

## 1. Я выбрала слово _замок_.
МАС говорит о его значениях следующее:
1. **за́мок**
  1. Дворец и крепость феодала. Средневековый замок. На скале видны развалины какого-то замка. Пушкин, Путешествие в Арзрум. || Большой помещичий дом затейливой архитектуры. С одной стороны Обломовка, с другой — княжеский замок, с широким раздольем барской жизни. И. Гончаров, Обломов.
  2. Устар. Здание тюрьмы. — Ослабели [арестанты], сидевши в замке, расслабли, а их ведут в самое пекло, — осуждал кого-то приказчик, обращаясь к подошедшему Нехлюдову. Л. Толстой, Воскресение. Пальтецо его вместе с мелочью в карманах так и осталось в тюремном замке. Федин, Необыкновенное лето.
2. **замо́к**
  1. Устройство для запирания чего-л. ключом. Дверной замок. Висячий замок. Запереть на замок. || Приспособление для смыкания, соединения концов чего-л. (цепочки, браслета и пр.).
  2. Воен. Приспособление для производства выстрела в некоторых видах огнестрельного оружия. Бойцы положили на траву, рядом с Пархоменко, пять пулеметов и замки с трех орудий. Вс. Иванов, Пархоменко.
  3. Тех. Способ скрепления бревен, брусьев и т. п. частей деревянных конструкций.
  4. Архит. Центральный камень в вершине свода, арки.

Я полагаю, что лучше всего будут классифицироваться наиболее частотные значения, а именно 1.1 - 'дворец' и 2.1 - 'устройство'.

Диагностические слова для значения 1.1: неприступный, король, фамильный

Диагностические слова для значения 2.1: ключ, дверь, щёлкнуть

In [0]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### Скачаем размеченный датасет и положим в пандас

In [11]:
!wget https://raw.githubusercontent.com/flying-bear/AutoT/master/hw/3/homonyms.txt

--2019-12-07 22:06:37--  https://raw.githubusercontent.com/flying-bear/AutoT/master/hw/3/homonyms.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/plain]
Saving to: ‘homonyms.txt’

homonyms.txt        100%[===================>]  12.04K  --.-KB/s    in 0s      

2019-12-07 22:06:37 (97.4 MB/s) - ‘homonyms.txt’ saved [12331/12331]



In [0]:
with open('homonyms.txt', 'r', encoding='utf-8') as f:
  lines = [line.strip().split('\t') for line in f.readlines()]

In [84]:
df = pd.DataFrame(lines, columns = ['sense', 'text']) 
df['sense'] = df['sense'].apply(int)
df['text'] = df['text'].apply(lambda s: s.strip())
df.head()

,sense,text
0,2,В этот момент щелкнул замок и дверь чуть приот...
1,2,"Мы поднялись, и я позвонил в дверь. Щелкнул за..."
2,2,"Андрей закрыл дверь, и никелированный замок гр..."
3,2,"Я зашёл в маленькую грязную комнату с лавкой, ..."
4,2,Дверь мягко щелкнула и ответила маслянистыми п...


##2. Я выбрала Адаграм

### Установим его

In [1]:
!wget https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib

--2019-12-07 21:59:56--  https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.42.78
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.42.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1462416741 (1.4G) [application/x-www-form-urlencoded]
Saving to: ‘all.a010.p10.d300.w5.m100.nonorm.slim.joblib’

all.a010.p10.d300.w 100%[===================>]   1.36G  16.3MB/s    in 87s     

2019-12-07 22:01:23 (16.0 MB/s) - ‘all.a010.p10.d300.w5.m100.nonorm.slim.joblib’ saved [1462416741/1462416741]



In [3]:
!pip install git+https://github.com/lopuhin/python-adagram.git

  Cloning https://github.com/lopuhin/python-adagram.git to /tmp/pip-req-build-m1xvfrrs
  Running command git clone -q https://github.com/lopuhin/python-adagram.git /tmp/pip-req-build-m1xvfrrs
  Created wheel for adagram: filename=adagram-0.0.1-cp36-cp36m-linux_x86_64.whl size=464608 sha256=0f23aadd7adc826bb099e6b4f896512f97d00c41c9482d8ced27096ea9f947c0
  Stored in directory: /tmp/pip-ephem-wheel-cache-smfngs_j/wheels/11/0f/46/f5df96670df8f7973b4c2311ffc9b02e435a7bd3207f992c4d
Successfully built adagram


In [2]:
!pip install pymystem3==0.1.10

### Imports

In [0]:
import adagram
import numpy as np
import re

from pymystem3 import Mystem

### Инициализация

In [0]:
mystem = Mystem()

In [0]:
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

In [0]:
def preprocess(text): 
  """
  cleans a text of punctuation and case
  
  :param text: str to be cleaned
  :return: str in lowercase and stripped of punctuation
  """
  low = text.lower().strip()
  stripped = re.sub('!|\.|,|#|$|%|\\|\'|\(|\)|-|\+|\*|/|\:|;|<|>|=|\?|\[|\]|@|^|_|`|{|}|~', '', low)
  return stripped

def lemmatize(text):
  """
  preprocesses and lemmatizes a string in mystem

  :param text: str to be lemmatized
  :return: list of str, lemmas
  """
  return mystem.lemmatize(preprocess(text))

### Посмотрим на значения слова замок в Адаграме

In [415]:
synonyms = {}
for i, prob in vm.word_sense_probs('замок'):
  num = i+1
  closest = [tr[0] for tr in vm.sense_neighbors('замок', i)]
  print(f'{num}: {", ".join(closest)}')
  synonyms[num] = closest

1: орденский, монсегюр, аббатство, дворец, сеньория, герцог, инстербург, каркассон, резиденция, амбуаз
2: задвижка, запор, засов, ключ, отмыкать, шпингалет, щеколда, защелка, запираться, дверь
3: крепость, башня, бастион, средневековый, величественный, донжон, цитадель, башня, здание, крепостной
4: крепость, дворец, башня, пещера, форт, холд, дом, подземелье, королевский, ругна


Выглядит так, будто значения 1, 3 и 4 относятся к нашему значению 1 ('дворец'), а значение 2 - к нашему значению 2 ('устройство'). Будем иметь это ввиду, определяя точность позднее. (Сравним!)

In [0]:
senses = {1: {1, 3, 4}, 
          2 : {2}}

### Приступим к разрешению омонимии

In [0]:
def disambiguate(model, word, context):
  """
  disambiguates a word providing a number that
  corresponds to the meaning of a word in a context using adagram

  :param model: adagram model, pretrained
  :param word: str, word to be disambiguated
  :param contaext: str, context of the word
  :return: int, number corresponding to the meaning of the word in the context
  """
  word, _ = lemmatize(word)
  probs = model.disambiguate(word, lemmatize(context))
  return 1 + probs.argmax()

In [85]:
df['predict_sense_id'] = [disambiguate(vm, 'замок', context) for context in df['text']]
df.head()

,sense,text,predict_sense_id
0,2,В этот момент щелкнул замок и дверь чуть приот...,2
1,2,"Мы поднялись, и я позвонил в дверь. Щелкнул за...",2
2,2,"Андрей закрыл дверь, и никелированный замок гр...",2
3,2,"Я зашёл в маленькую грязную комнату с лавкой, ...",2
4,2,Дверь мягко щелкнула и ответила маслянистыми п...,2


## 3. Припишем номера значений словарным толкованиям

### Значение 1 'дворец'

In [57]:
def1 = 'Дворец и крепость феодала. Средневековый замок. На скале видны развалины какого-то замка. Пушкин, Путешествие в Арзрум. || Большой помещичий дом затейливой архитектуры. С одной стороны Обломовка, с другой — княжеский замок, с широким раздольем барской жизни. И. Гончаров, Обломов.'
sense_def1 = disambiguate(vm, 'замок', def1)
sense_def1

3

### Значение 2 'устройство'

In [58]:
def2 = 'Устройство для запирания чего-л. ключом. Дверной замок. Висячий замок. Запереть на замок. || Приспособление для смыкания, соединения концов чего-л. (цепочки, браслета и пр.).'
sense_def2 = disambiguate(vm, 'замок', def2)
sense_def2

2

### Запишем эти числа в датасет

In [86]:
df['auto_sense'] = df['sense'].apply(lambda x: 2 if x == 2 else 3)
df.head()

,sense,text,predict_sense_id,auto_sense
0,2,В этот момент щелкнул замок и дверь чуть приот...,2,2
1,2,"Мы поднялись, и я позвонил в дверь. Щелкнул за...",2,2
2,2,"Андрей закрыл дверь, и никелированный замок гр...",2,2
3,2,"Я зашёл в маленькую грязную комнату с лавкой, ...",2,2
4,2,Дверь мягко щелкнула и ответила маслянистыми п...,2,2


## 4. Соотнесём значения и толкования с помощью тезауруса

In [14]:
!pip install wiki_ru_wordnet

In [0]:
from wiki_ru_wordnet import WikiWordnet

In [0]:
ruwn = WikiWordnet()

In [0]:
def get_words_of_synset(syns):
  return [word.lemma() for word in syns.get_words()]

### Посмотрим, какие значения слова _замок_ выглядят wiki_ru_wordnet:

In [448]:
syns_meanings = {}
for i, s in enumerate(ruwn.get_synsets('замок')):
  words = get_words_of_synset(s)
  syns_meanings[i] = words 
  print(i, ', '.join(words))

0 крепость, цитадель, замок
1 замок
2 замок, запор
3 замок
4 замок
5 замок
6 замок
7 замок


Не очень понятно, что это значит (те, к которым нет синонимов!), посмотрим на гиперонимы. (И добавим их к нашему представлению о значениях в wiki_ru_wordnet.)

In [0]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [449]:
for i, s in enumerate(ruwn.get_synsets('замок')):
  hypernyms = ruwn.get_hypernyms(s)
  if not hypernyms:
    print(i, 'no hypernyms')
  else:
    syns_meanings[i] += flatten([get_words_of_synset(h) for h in hypernyms])
    print(i, [', '.join(get_words_of_synset(h)) for h in hypernyms])

0 ['дворец', 'жильё, дом, жилплощадь, жилище, резиденция']
1 no hypernyms
2 ['устройство, инструмент, девайс, приспособление', 'приспособление']
3 no hypernyms
4 ['способ, метод, метода']
5 no hypernyms
6 no hypernyms
7 no hypernyms


### Посмотрим, как выглядят определения:

In [248]:
ex_def = list(ruwn.get_synsets('замок')[2].get_words())[0].definition()
ex_def

'замок~ru~замок~ru~[[устройство]] для запирания чего-либо [[ключ]]ом {{пример|Дверной {{выдел|замок}}.}} {{пример|Запереть дверь на {{выдел|замок}}.}} {{пример|Ключ в {{выдел|замке}}.}} {{пример|Каморка запиралась на {{выдел|замо́к}}, напоминавший своим видом калач, только чёрный; ключ от этого {{выдел|замка́}} Герасим всегда носил с собой на пояске.|Тургенев|[[:s:Муму (Тургенев)|Муму]]|1852|источник=source}} {{пример|Ключ заскрипел в заржавленном {{выдел|замке}}, и дверь отворилась; им в лицо пахнуло сыростью.|Лермонтов|[[:s:Штосс (Лермонтов)|Штосс]]|1841|источник=source}}~57866~29025'

Не слишком-то красиво. Прийдётся чистить.
Сначала почистим примеры.

In [0]:
ex_example = '|Каморка запиралась на |замо́к}}, напоминавший своим видом калач, только чёрный; ключ от этого |замка́}} Герасим всегда носил с собой на пояске.|Тургенев|[[:s:Муму (Тургенев)|Муму]]|1852|источник=source}} {{'

In [0]:
def clean_ruwn_example(example):
  # Герасим всегда носил с собой на пояске.|Тургенев|[[:s:Муму (Тургенев)|Муму]]|1852|источник=source}}
  # пред ним стояла Настасья Филипповна.|Ф. М. Достоевский|Идиот|1869|источник=НКРЯ
  if '.|' in example:
    text, meta = example.split('.|')
    text += '.'
    text = re.sub('!|#|$|%|\\|\'|\+|\*|/|\:|;|<|>|=|\?|\[|\]|@|^|_|`|{|}|~|\||[0-9]', '', text)
    text = text.strip().strip('|')
    text = text.capitalize()
    meta = meta.strip(' {{')
    meta = meta.strip('}}')
    meta_data = meta.split('|')
    if len(meta_data) == 5:  # если есть все мета данные, мы знаем, как они структурированы
      auth, name, proper_name, year, sorce = meta_data 
      return text + ' (Из "' + proper_name.strip(']]') + '", ' + auth + ')'
    elif '|[[' not in meta and len(meta_data) == 4:
      auth, proper_name, year, sorce = meta_data
      return text + ' (Из "' + proper_name.strip(']]') + '", ' + auth + ')'
    else:
      return text
  else:
    return re.sub('!|#|$|%|\\|\'|\+|\*|/|\:|;|<|>|=|\?|\[|\]|@|^|_|`|{|}|~|\||[0-9]', '', example).strip().strip('|').capitalize()

In [353]:
clean_ruwn_example(ex_example)

'Каморка запиралась на замо́к, напоминавший своим видом калач, только чёрный ключ от этого замка́ герасим всегда носил с собой на пояске. (Из "Муму", Тургенев)'

Теперь почистим всё определение целиком.

In [0]:
def clean_ruwn_definition(definition, word):
  definition = re.sub(f'{word}~ru~{word}~ru~', f'{word.capitalize()} - ', definition)  # уберём начальный тег
  definition = re.sub('\xa0', '', definition)
  for pair, specific in  re.findall('({{(.+?)\|ru}})', definition):  # обработаем узус {{плотн.|ru}}, заменим его на (плотн.)
    pair = re.sub('\|', '\\\|', pair)  # защитим | с помощью \
    definition = re.sub(pair, f'({specific})', definition)
  for pair, replacement in re.findall('(\[\[[А-Яа-яёЁ]+?\|([А-Яа-яёЁ]+?)\]\])', definition):  # уберём штуки вроде [[конец|концов]], где нужен только второй вариант
    pair = re.sub('\[', '\\\[', pair)  # защитим [ с помощью \
    pair = re.sub('\|', '\\\|', pair)  # защитим | с помощью \ 
    definition = re.sub(pair, replacement, definition)  # заменим защищённое выражение
  definition = re.sub('{{выдел|', '',definition)  # заменим выделения типа {{выдел|замок}} в примере на 'замок'
  definition, *examples = definition.split('пример')
  examples = [clean_ruwn_example(example) for example in examples]
  definition = re.sub('!|#|$|%|\\|\'|\+|\*|/|\:|;|<|>|=|\?|\[|\]|@|^|_|`|{|}|~|\||[0-9]', '', definition)
  definition = definition.strip() + '.'
  return definition, examples

In [333]:
clean_ruwn_definition(ex_def, 'замок')

('Замок - устройство для запирания чего-либо ключом.',
 ['Дверной замок.',
  'Запереть дверь на замок.',
  'Ключ в замке.',
  'Каморка запиралась на замо́к, напоминавший своим видом калач, только чёрный ключ от этого замка́ герасим всегда носил с собой на пояске. (Из "Муму", Тургенев)',
  'Ключ заскрипел в заржавленном замке, и дверь отворилась им в лицо пахнуло сыростью.'])

### Посмотрим на определения всех значений wiki_ru_wordnet

In [354]:
for i, s in enumerate(ruwn.get_synsets('замок')):
  for j, w in enumerate(list(s.get_words())):
    word = w.lemma()
    defenition, examples = clean_ruwn_definition(w.definition(), word)
    print(i, j, defenition)
    if examples != ['']:
      print('    примеры:')
      for ex in examples:
        print(f'\t- {ex}')

0 0 Крепость - военн. укреплённое место с долговременными оборонительными сооружениями, приспособленное для круговой обороны.
    примеры:
	- Крепость оказалась, как крепость,-четырехугольник стен с восемью фланкирующими башнями стены, сделанные из твёрдой глины, были толщиною в четыре сажени при такой же, четырёхсаженной, высоте по отвесу кроме того, на гребне стен торчали зубцы вышиною в полсажени. огромный ров был выкопан перед стенами, но артиллерия крепости была слаба,— всего три орудия гарнизон тоже невелик четыреста человек. (Из "Севастопольская страда", С.Н.Сергеев-Ценский)
0 1 Цитадель - центральное, наиболее сильно фортифицированное укрепление в крепости или городе.
    примеры:
	- Между цитаделью и городскими строениями находилось открытое незастроенное пространство, которое называлось эспланадой.
0 2 Замок - здание (или комплекс зданий), обычно обнесённое стеной и сочетающее в себе оборонительную и жилую функции (в наиболее распространённом значении — укреплённое жилище фео

Чтобы было больше шансов сопоставить значения из adagram значениям из wiki_ru_wordnet, расширим словарь синонимов adagram с пмощью всех синсетов для каждого из изначальных синонимов.

In [437]:
broader_synonyms = {1: set(),
                    2: set(),
                    3: set(),
                    4: set()}
for meaning_id in synonyms:
  for synonym in synonyms[meaning_id]:
    broader_synonyms[meaning_id].add(synonym)
    for synset in ruwn.get_synsets(synonym): 
      lemmas = get_words_of_synset(synset)
      if lemmas:
        broader_synonyms[meaning_id].update(set(lemmas))

for key in broader_synonyms:
  print(key, ', '.join(broader_synonyms[key]))

1 жилплощадь, орденский, монсегюр, амбуаз, аббатство, жильё, дворец, жилище, дом, сеньория, каркассон, герцог, инстербург, резиденция
2 запираться, замок, щеколда, шпингалет, жужик, задвижка, дверь, отмыкать, ключ, родник, запор, засов, источник, защелка, шкет
3 донжон, величественный, постройка, бастион, башня, крепость, цитадель, корпус, замок, стойкость, средневековый, крепостной, дом, крепкость, строение, здание, каланча, точечный дом
4 каланча, королевский, замок, жилище, пещера, подземелье, холд, дворец, строение, точечный дом, постройка, корпус, башня, крепость, цитадель, жилплощадь, стойкость, форт, вертеп, ругна, жильё, крепкость, здание, дом, резиденция


### Попробуем теперь подсчитать, какие номера значений больше всего пересекаются в списке синонимов. Их мы и объявим тождественными.

In [466]:
mapping = {1: [],
           2: [],
           3: [],
           4: []}
for meaning_id in broader_synonyms:
  synonyms = broader_synonyms[meaning_id]
  for syns_meaning_id in syns_meanings:
    syns_synonyms = set(syns_meanings[syns_meaning_id])
    similarity = len(synonyms.intersection(syns_synonyms))
    mapping[meaning_id].append(similarity)

for key, value in mapping.items():
  print(key, value)

for meaning_id in mapping:
  mapping[meaning_id] = np.argmax(mapping[meaning_id])

for key, value in mapping.items():
  print(f'Значение {key} в adagram приблизительно соответствует значению {value} в wiki_ru_wordnet')

1 [6, 0, 0, 0, 0, 0, 0, 0]
2 [1, 1, 2, 1, 1, 1, 1, 1]
3 [4, 1, 1, 1, 1, 1, 1, 1]
4 [9, 1, 1, 1, 1, 1, 1, 1]
значение 1 в adagram приблизительно соответствует значению 0 в wiki_ru_wordnet
значение 2 в adagram приблизительно соответствует значению 2 в wiki_ru_wordnet
значение 3 в adagram приблизительно соответствует значению 0 в wiki_ru_wordnet
значение 4 в adagram приблизительно соответствует значению 0 в wiki_ru_wordnet


### Выведем их определения.

In [470]:
print(f'Значениям 1, 3 и 4 adagram соответсвует значение 0 в wiki_ru_wordnet. Его определение таково:')
for w in ruwn.get_synsets('замок')[0].get_words():
  word = w.lemma()
  if word == 'замок':
    defenition, examples = clean_ruwn_definition(w.definition(), word)
    print(' ', defenition)
    if examples != ['']:
      print('    примеры:')
      for ex in examples:
        print(f'\t- {ex}')
print('\n')
print(f'Значению 2 в adagram соответсвует значение 2 в wiki_ru_wordnet. Его определение таково:')
for w in ruwn.get_synsets('замок')[2].get_words():
  word = w.lemma()
  if word == 'замок':
    defenition, examples = clean_ruwn_definition(w.definition(), word)
    print(' ', defenition)
    if examples != ['']:
      print('    примеры:')
      for ex in examples:
        print(f'\t- {ex}')

Значениям 1, 3 и 4 adagram соответсвует значение 0 в wiki_ru_wordnet. Его определение таково:
  Замок - здание (или комплекс зданий), обычно обнесённое стеной и сочетающее в себе оборонительную и жилую функции (в наиболее распространённом значении — укреплённое жилище феодала в средневековой Европе).
    примеры:
	- Почтенный за́мок был построен,  как замки строиться должны  отменно прочен и спокоен  во вкусе умной старины. (Из "Евгений Онегин. Глава 2", Пушкин)


Значению 2 в adagram соответсвует значение 2 в wiki_ru_wordnet. Его определение таково:
  Замок - устройство для запирания чего-либо ключом.
    примеры:
	- Дверной замок.
	- Запереть дверь на замок.
	- Ключ в замке.
	- Каморка запиралась на замо́к, напоминавший своим видом калач, только чёрный ключ от этого замка́ герасим всегда носил с собой на пояске. (Из "Муму", Тургенев)
	- Ключ заскрипел в заржавленном замке, и дверь отворилась им в лицо пахнуло сыростью. (Из "Штосс", Лермонтов)


Как и было сказано до того, значения 1, 3 и 4 adagram на самом деле примерно одинаковые.

## 5. Подсчитаем точность

In [87]:
compare_strict = lambda dataframe: 1 if dataframe['auto_sense'] == dataframe['predict_sense_id'] else 0
df['compare_strict'] = df.apply(compare_strict, axis=1)
compare = lambda dataframe: 1 if  dataframe['predict_sense_id'] in senses[dataframe['sense']] else 0
df['compare'] = df.apply(compare, axis=1)
df.tail()

,sense,text,predict_sense_id,auto_sense,compare_strict,compare
55,1,"К примеру, привидение, обитающее в фамильном з...",1,3,0,1
56,1,"Удивительно, но только пару лет назад Ольга см...",1,3,0,1
57,1,"Он чокнулся с Астросом и Зарецким, поклонился ...",4,3,0,1
58,1,Я обнаружила в себе совершенно не свойственный...,4,3,0,1
59,1,"Продав все свои должности, он удалился в свой ...",1,3,0,1


Если считать строго, то значение, определённое автоматически неверно, однако значения очень близки, и compare отражает это.

In [92]:
accuracy_strict = np.mean(df['compare_strict'])
accuracy = np.mean(df['compare'])
print(f'strict:\t\t{accuracy_strict}\nnon-strict:\t{accuracy}')

strict:		0.55
non-strict:	0.9
